## Практическая работа №6
##### Сиразетдинов Рустем (Вариант - 20)

Предсказание стоимости медецинской страховки

Данные: https://www.kaggle.com/mirichoi0218/insurance

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
from plotly.offline import plot, iplot, init_notebook_mode
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.decomposition import PCA
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.metrics import r2_score, mean_squared_error
import warnings
warnings.filterwarnings("ignore")
import math

# Выгрузка датасета
ins = pd.read_csv('C:\\Users\\averu\\Documents\\git_local\\programming-practice\\IDA\\IDA-practice-5\\insurance.csv')
print("There are {:,} observations and {} columns in the data set.".format(ins.shape[0], ins.shape[1]))
print("There are {} missing values in the data.".format(ins.isna().sum().sum()))


ins['sex'] = ins['sex'].str.capitalize()
ins['smoker'] = ins['smoker'].apply(lambda x: 'Smoker' if x=='yes' else 'Non-Smoker')
ins['region'] = ins['region'].str.capitalize()

In [ ]:
init_notebook_mode(connected=True)
px.defaults.template = "plotly_white"
plot_df=ins.copy()
fig = px.box(plot_df, x="region", y="charges", color="region", 
             notched=True, points="outliers", height=600,
             title="",
             color_discrete_sequence=['#B14B51', '#D0A99C', '#5D8370', '#6C839B'])
fig.update_traces(marker=dict(size=9, opacity=0.5, line=dict(width=1,color="#F7F7F7")), showlegend=False)
fig.update_layout(font_color="#303030", xaxis_title='Region', yaxis_title='Claim Amount, $',
                  yaxis=dict(showgrid=True, gridwidth=1, gridcolor='#EAEAEA', zerolinecolor='#EAEAEA'))
fig.show()

Для каждого региона графики показывают, что страховые сборы положительно искажены с несколькими большими отклонениями. Страховые сборы более изменчивы в Юго-Восточном регионе, который содержит самую высокую претензию в наборе данных - более 63 000 долларов США, а также самую низкую претензию - 1121 доллар США. Северо-Восточный регион имеет самую высокую среднюю стоимость в целом, хотя, поскольку отметки в графиках накладываются друг на друга, средние суммы претензий, вероятно, существенно не отличаются.

In [ ]:
ins['female'] = ins['sex'].apply(lambda x: 1 if x=='Female' else 0)
ins['smoker_yes'] = ins['smoker'].apply(lambda x: 1 if x=='Smoker' else 0)
ins.drop(['sex', 'smoker'], axis=1, inplace=True)

sns.set_context("notebook")
fig, ax = plt.subplots(figsize=(9,6))   
corr=ins.corr()
mask=np.triu(np.ones_like(corr, dtype=bool))[1:, :-1]
corr=corr.iloc[1:,:-1].copy()
ax=sns.heatmap(corr, mask=mask, vmin=-.1, vmax=.9, center=0, annot=True, fmt='.2f', 
               cmap='ocean', linewidths=4, annot_kws={"fontsize":12})
ax.set_title('', fontsize=18)
ax.set_xticklabels(ax.get_xticklabels(), rotation=45, horizontalalignment='right',fontsize=12)
ax.set_yticklabels(ax.get_yticklabels(), fontsize=12)
fig.show()

Корреляционная матрица показывает, что курильщики имеют сильную положительную связь со страховыми взносами на уровне 0,79, но в целом переменные в наборе данных не слишком сильно коррелируют друг с другом.

Построение линейной регрессии

In [ ]:
data = pd.read_csv('C:\\Users\\averu\\Documents\\git_local\\programming-practice\\IDA\\IDA-practice-5\\insurance.csv')

data['sex'] = data['sex'].str.capitalize()
data['smoker'] = data['smoker'].apply(lambda x: 'Smoker' if x=='yes' else 'Non-Smoker')

data['female'] = data['sex'].apply(lambda x: 1 if x=='Female' else 0)
data['smoker_yes'] = data['smoker'].apply(lambda x: 1 if x=='Smoker' else 0)


region = pd.get_dummies(data['region']) 
data = pd.concat((data, region), axis=1)
data = data.loc[:, data.columns.isin(['age', 'female', 'bmi', 'children' , 'smoker_yes', 'charges', 'northeast', 'northwest','southeast', 'southwest'])]

data.head(7)

Модель 1. Все признаки присутствуют.

In [ ]:
y_train=data.pop('charges')
X_train = data.copy()

import statsmodels.api as sm
X_train_sm = sm.add_constant(X_train)

lm= sm.OLS(y_train, X_train_sm).fit()

lm.summary()

Модель 2. Квадрат признака `bmi`. 

In [ ]:
# y_train=data.pop('charges')
X_train = data.copy()
for i in range(0 , len(X_train['bmi'])):
    X_train['bmi'][i] = X_train['bmi'][i] * X_train['bmi'][i]

# X_train['bmi'] = list(map(lambda x: x, X_train['bmi']))

import statsmodels.api as sm
X_train_sm = sm.add_constant(X_train)

bmi_sq= sm.OLS(y_train, X_train_sm).fit()

bmi_sq.summary()

Модель 3. Квадрат признака `age`.

In [ ]:
# y_train=data.pop('charges')
X_train = data.copy()
for i in range(0 , len(X_train['age'])):
    X_train['age'][i] = X_train['age'][i] * X_train['age'][i]


import statsmodels.api as sm
X_train_sm = sm.add_constant(X_train)

age_sq= sm.OLS(y_train, X_train_sm).fit()

age_sq.summary()

Модель 4. Квадраты признаков `age` и `bmi`.

In [ ]:
# y_train=data.pop('charges')
X_train = data.copy()
for i in range(0 , len(X_train['age'])):
    X_train['age'][i] = X_train['age'][i] * X_train['age'][i]
    X_train['bmi'][i] = X_train['bmi'][i] * X_train['bmi'][i]


import statsmodels.api as sm
X_train_sm = sm.add_constant(X_train)

bmi_age_sq= sm.OLS(y_train, X_train_sm).fit()

bmi_age_sq.summary()

Модель 5.

In [ ]:
# y_train=data.pop('charges')
X_train = data.copy()
for i in range(0 , len(X_train['age'])):
    X_train['age'][i] = X_train['age'][i] * X_train['age'][i]
    X_train['bmi'][i] = X_train['bmi'][i] * X_train['bmi'][i]


import statsmodels.api as sm
X_train_sm = sm.add_constant(X_train)

lm_5= sm.OLS(y_train, X_train_sm).fit()
lm_5.summary()

In [ ]:
# y_train=data.pop('charges')

X_train = data.copy()
for i in range(0 , len(X_train['age'])):
    X_train['age'][i] = math.log(X_train['age'][i],10)
    


import statsmodels.api as sm
X_train_sm = sm.add_constant(X_train)

age_lg= sm.OLS(y_train, X_train_sm).fit()
age_lg.summary()

In [ ]:
# y_train=data.pop('charges')
import math
X_train = data.copy()
for i in range(0 , len(X_train['bmi'])):
    X_train['bmi'][i] = math.log(X_train['bmi'][i],10)
    


import statsmodels.api as sm
X_train_sm = sm.add_constant(X_train)

bmi_lg= sm.OLS(y_train, X_train_sm).fit()
bmi_lg.summary()

In [ ]:
# y_train=data.pop('charges')

X_train = data.copy()
for i in range(0 , len(X_train['bmi'])):
    X_train['bmi'][i] = math.log(X_train['bmi'][i],10)
    X_train['age'][i] = math.log(X_train['age'][i],10)


import statsmodels.api as sm
X_train_sm = sm.add_constant(X_train)

bmi_age_lg= sm.OLS(y_train, X_train_sm).fit()
bmi_age_lg.summary()

In [ ]:

X_train = data.copy()
for i in range(0 , len(X_train['bmi'])):
    X_train['bmi'][i] = math.log(X_train['bmi'][i],10)
    X_train['age'][i] = X_train['age'][i] * X_train['age'][i]


import statsmodels.api as sm
X_train_sm = sm.add_constant(X_train)

lm_16= sm.OLS(y_train, X_train_sm).fit()
lm_16.summary()

In [ ]:
# y_train=data.pop('charges')


X_train = data.loc[:, data.columns.isin(['bmi', 'smoker_yes',])]

import statsmodels.api as sm
X_train_sm = sm.add_constant(X_train)

lm_9= sm.OLS(y_train, X_train_sm).fit()
lm_9.summary()

In [ ]:
# y_train=data.pop('charges')

X_train = data.loc[:, data.columns.isin(['age', 'smoker_yes',])]


import statsmodels.api as sm
X_train_sm = sm.add_constant(X_train)

age_smoke= sm.OLS(y_train, X_train_sm).fit()
age_smoke.summary()

In [ ]:
# y_train=data.pop('charges')
X_train = data.copy()
for i in range(0 , len(X_train['smoker_yes'])):
    X_train['smoker_yes'][i] = X_train['smoker_yes'][i] * X_train['smoker_yes'][i]


import statsmodels.api as sm
X_train_sm = sm.add_constant(X_train)

lm_11= sm.OLS(y_train, X_train_sm).fit()
lm_11.summary()

In [ ]:
# y_train=data.pop('charges')
X_train = data.loc[:, data.columns.isin(['age', 'bmi',])]


import statsmodels.api as sm
X_train_sm = sm.add_constant(X_train)

lm_12= sm.OLS(y_train, X_train_sm).fit()
lm_12.summary()

In [ ]:
# y_train=data.pop('charges')
y_train=data.pop('smoker_yes')
# X_train = [data['female'].copy(), data['bmi'].copy()]
X_train = data.loc[:, data.columns.isin(['age', 'female', 'bmi', 'children' , 'charges', 'northeast', 'northwest','southeast', 'southwest'])]

import statsmodels.api as sm
X_train_sm = sm.add_constant(X_train)

lm_13= sm.OLS(y_train, X_train_sm).fit()
lm_13.summary()

In [ ]:
# y_train=data.pop('charges')
y_train=data.pop('bmi')
# X_train = [data['female'].copy(), data['bmi'].copy()]
X_train = data.loc[:, data.columns.isin(['age', 'female', 'smoker_yes', 'children' , 'charges', 'northeast', 'northwest','southeast', 'southwest'])]

import statsmodels.api as sm
X_train_sm = sm.add_constant(X_train)

lm_20= sm.OLS(y_train, X_train_sm).fit()
lm_20.summary()

In [ ]:
# Dataframe to save results


regions = ins.region.unique()
s = StandardScaler()

actuals=[]
preds=[]
rmses=[]
r2_scores=[]
adj_r2_scores=[]

for i in regions:
    
    # Filter data by region
    print("\nRegion: {}\n".format(i))
    ins_df = ins[ins.region==i]
    X=ins_df.drop(['charges', 'region'], axis=1)
    y=ins_df.charges
    
    # Add polynomial features
    pf = PolynomialFeatures(degree=2, include_bias=False)
    X_pf = pd.DataFrame(data=pf.fit_transform(X), columns=pf.get_feature_names(X.columns))
    
    # Create training and test sets
    X_train, X_test, y_train, y_test = train_test_split(X_pf, y, test_size=0.2, random_state=1)
    
    # Scale features
    X_train_scaled = s.fit_transform(X_train)
    X_test_scaled = s.transform(X_test)
    
    # PCA
    pca = PCA(.95)
    X_train_pca=pca.fit_transform(X_train_scaled)
    X_test_pca=pca.transform(X_test_scaled)
    print("Number of Principal Components = {}".format(pca.n_components_))
    print("Train Shape:{} {}  Test Shape:{} {}".format(X_train_pca.shape, y_train.shape, X_test_pca.shape, y_test.shape))
    
    # Linear Regression
    lr = LinearRegression().fit(X_train_pca, y_train)
    y_pred=lr.predict(X_test_pca)
    rmse=np.sqrt(mean_squared_error(y_test, y_pred)).round(2)
    r2=r2_score(y_test, y_pred)
    adj_r2 = 1 - (1-r2)*(len(y_test)-1)/(len(y_test)-X_test_pca.shape[1]-1)
    
    actuals.append(pd.Series(y_test, name='actuals').reset_index())
    preds.append(pd.Series(y_pred, name='preds').reset_index(drop=True))
    rmses.append(rmse)
    r2_scores.append(r2)
    adj_r2_scores.append(adj_r2)
    
    print("Test Error (RMSE) = {:,}".format(rmse))
    print("R-Squared = {:.2f}%, Adjusted R-Squared = {:.2f}%".format(r2*100, adj_r2*100))
    if i != 'Northeast':
        print("__________________________")

# Plot results
for i in range(0,4):
    actuals[i].loc[:,'index']=regions[i]
actual = pd.concat([actuals[i] for i in range(4)], axis = 0)
pred = pd.concat([preds[i] for i in range(4)], axis = 0)
df = pd.concat([actual, pred], axis=1).reset_index(drop=True)
col = ["#B14B51", '#D0A99C', '#5D8370','#6C839B']
fig = px.scatter(df, x="actuals", y="preds", color="index", trendline="ols", height=700,
                 title="Actual vs Predicted Insurance Costs by Region,<br>Linear Regression with Principal Component Analysis",
                 color_discrete_sequence=col, opacity=0.7, facet_col='index', facet_col_wrap=2)

fig.for_each_annotation(lambda a: a.update(text=a.text.split("=")[-1]))
fig.add_annotation(text="Adj. R-Squared = {:.1f}%<br>RMSE = {:,.0f}".format(adj_r2_scores[0]*100,rmses[0]),
                   x=51e3,y=15e3, row=2,col=1, showarrow=False)
fig.add_annotation(text="Adj. R-Squared = {:.1f}%<br>RMSE = {:,.0f}".format(adj_r2_scores[1]*100,rmses[1]),
                   x=51e3,y=15e3, row=2,col=2, showarrow=False)
fig.add_annotation(text="Adj. R-Squared = {:.1f}%<br>RMSE = {:,.0f}".format(adj_r2_scores[2]*100,rmses[2]),
                   x=51e3,y=15e3, row=1,col=1, showarrow=False)
fig.add_annotation(text="Adj. R-Squared = {:.1f}%<br>RMSE = {:,.0f}".format(adj_r2_scores[3]*100,rmses[3]),
                   x=51e3,y=15e3, row=1,col=2, showarrow=False)

fig.update_traces(hovertemplate="Actual Cost: %{x:$,.2f}<br>Predicted Cost: %{y:$,.2f}",
                  marker=dict(size=10, line=dict(width=1,color="#F7F7F7")),
                  selector=dict(mode="markers"), showlegend=False)
fig.update_xaxes(title="Actual Cost, $", row=1)
fig.update_xaxes(showgrid=True, gridwidth=1, gridcolor='#EAEAEA',
                 zeroline=True, zerolinewidth=2, zerolinecolor='#5E5E5E')
fig.update_yaxes(title="Predicted Cost, $", col=1)
fig.update_yaxes(showgrid=True, gridwidth=1, gridcolor='#E3E3E3',
                 zeroline=True, zerolinewidth=2, zerolinecolor='#5E5E5E')
fig.update_layout(font_color="#303030", paper_bgcolor="white", plot_bgcolor="white")
fig.show()

Gradient Boosting

In [ ]:
actuals=[]
preds=[]
rmses=[]
r2_scores=[]
adj_r2_scores=[]
feat_importance=pd.DataFrame()
regions = ins.region.unique()
s = StandardScaler()
col = ["#B14B51", '#D0A99C', '#5D8370','#6C839B']

for i in regions:
    
    # Разделение датасета по региону
    ins_df = ins[ins.region==i]
    X=ins_df.drop(['charges', 'region'], axis=1)
    y=ins_df.charges
    
    
    
    pf = PolynomialFeatures(degree=2, include_bias=False)
    X_pf = pd.DataFrame(data=pf.fit_transform(X), columns=pf.get_feature_names(X.columns))
    
    # Распределение на выборки
    X_train, X_test, y_train, y_test = train_test_split(X_pf, y, test_size=0.2, random_state=1)
    X_train = pd.DataFrame(X_train, columns = X_pf.columns)
    X_test = pd.DataFrame(X_test, columns = X_pf.columns)
    actuals.append(pd.Series(y_test, name='actuals').reset_index())
    print("\nRegion: {}\n".format(i))
    print("Train Shape:{} {}  Test Shape:{} {}".format(X_train.shape, y_train.shape, X_test.shape, y_test.shape))
    
    
    X_train = pd.DataFrame(data=s.fit_transform(X_train), columns=X_pf.columns)
    X_test = pd.DataFrame(data=s.transform(X_test), columns=X_pf.columns)
    
    
    grid = {'learning_rate': [0.0001, 0.001, 0.01, 0.1, 0.25, 0.5],
            'n_estimators': [int(x) for x in np.linspace(start = 200, stop = 1000, num = 5)],
            'subsample': [0.5, 0.8, 1],
            'min_samples_split': [2, 5, 10],
            'min_samples_leaf': [1, 2, 4],
            'max_depth': [int(x) for x in np.linspace(2, 10, num = 5)],
            'max_features': [None, 'sqrt']}
    xgb=GradientBoostingRegressor(random_state=21)
    xgb_cv=RandomizedSearchCV(estimator=xgb, param_distributions=grid, scoring='neg_mean_squared_error', 
                              n_iter=100, cv=3, random_state=21, n_jobs=-1)
    xgb_cv.fit(X_train, y_train)
    y_pred=xgb_cv.predict(X_test)
    preds.append(pd.Series(y_pred, name='preds').reset_index(drop=True))
    rmse=np.sqrt(mean_squared_error(y_test, y_pred)).round(2)
    r2=r2_score(y_test, y_pred)
    adj_r2 = 1 - (1-r2)*(len(y_test)-1)/(len(y_test)-X_test.shape[1]-1)
    rmses.append(rmse)
    r2_scores.append(r2)
    adj_r2_scores.append(adj_r2)
    
    
    feat_importance["Importance_"+str(i)]=xgb_cv.best_estimator_.feature_importances_
    
    print("Test Error (RMSE) = {:,}".format(rmse))
    print("R-Squared = {:.2f}%, Adjusted R-Squared = {:.2f}%".format(r2*100, adj_r2*100))
    if i != 'Northeast':
        print("__________________________")

# Plot results
for i in range(0,4):
    actuals[i].loc[:,'index']=regions[i]
actual = pd.concat([actuals[i] for i in range(4)], axis = 0)
pred = pd.concat([preds[i] for i in range(4)], axis = 0)
df = pd.concat([actual, pred], axis=1).reset_index(drop=True)

fig = px.scatter(df, x="actuals", y="preds", color="index", trendline="lowess", height=700,
                 title="Реальные vs Предсказанные результаты стоимости соответсвтующих регионам,<br>Gradient Boosting",
                 color_discrete_sequence=col, opacity=0.7, facet_col='index', facet_col_wrap=2)

fig.for_each_annotation(lambda a: a.update(text=a.text.split("=")[-1]))
fig.add_annotation(text="Adj. R-Squared = {:.1f}%<br>RMSE = {:,.0f}".format(adj_r2_scores[0]*100,rmses[0]),
                   x=51e3,y=15e3, row=2,col=1, showarrow=False)
fig.add_annotation(text="Adj. R-Squared = {:.1f}%<br>RMSE = {:,.0f}".format(adj_r2_scores[1]*100,rmses[1]),
                   x=51e3,y=15e3, row=2,col=2, showarrow=False)
fig.add_annotation(text="Adj. R-Squared = {:.1f}%<br>RMSE = {:,.0f}".format(adj_r2_scores[2]*100,rmses[2]),
                   x=51e3,y=15e3, row=1,col=1, showarrow=False)
fig.add_annotation(text="Adj. R-Squared = {:.1f}%<br>RMSE = {:,.0f}".format(adj_r2_scores[3]*100,rmses[3]),
                   x=51e3,y=15e3, row=1,col=2, showarrow=False)

fig.update_traces(hovertemplate="Actual Cost: %{x:$,.2f}<br>Predicted Cost: %{y:$,.2f}",
                  marker=dict(size=10,line=dict(width=1,color="#F7F7F7")),
                  selector=dict(mode="markers"), showlegend=False)
fig.update_xaxes(title="Actual Cost, $", row=1)
fig.update_xaxes(showgrid=True, gridwidth=1, gridcolor='#EAEAEA',
                 zeroline=True, zerolinewidth=2, zerolinecolor='#5E5E5E')
fig.update_yaxes(title="Predicted Cost, $", col=1)
fig.update_yaxes(showgrid=True, gridwidth=1, gridcolor='#E3E3E3',
                 zeroline=True, zerolinewidth=2, zerolinecolor='#5E5E5E')
fig.update_layout(font_color="#303030", paper_bgcolor="white", plot_bgcolor="white")
fig.show()

In [ ]:
# models=["Linear Regression", "KNN", "SVM", "Gradient Boosting"]
# mod_res=pd.DataFrame(columns=["Average RMSE", "Avg. Adjusted R2"], index=models)

# mod_res.iloc[3,0]=pd.Series(rmses).mean()
# mod_res.iloc[3,1]=pd.Series(adj_r2_scores).mean()
# mod_res["Average RMSE"]=mod_res["Average RMSE"].map('{:,.2f}'.format)
# mod_res["Avg. Adjusted R2"]=mod_res["Avg. Adjusted R2"].mul(100).map('{:.2f}%'.format)
# display(mod_res.sort_values("Average RMSE"))

In [ ]:
col=sns.color_palette("magma", 20).as_hex()[::-1]
feat_importance.set_index(X_train.columns, inplace=True)
ft=pd.DataFrame({"Средняя значимость":feat_importance.mean(axis=1)})
plot_df=ft.nlargest(20, columns="Средняя значимость").sort_values(by="Средняя значимость",ascending=False)
fig = px.bar(plot_df, x="Средняя значимость", y=plot_df.index, text="Средняя значимость", height=700,
             color=plot_df.index,width=700,opacity=.8,color_discrete_sequence=col)
fig.update_traces(texttemplate='%{text:.2f}', textposition='outside',
                  marker_line=dict(width=1, color='#3F3B3A'), showlegend=False, 
                  hovertemplate='Значимость = <b>%{x:.2}</b>')
fig.update_layout(title_text='Значимость факторов при оценке стоимости медицинской страховки', 
                  coloraxis_showscale=False, yaxis_title="", font_color="#303030", yaxis_linecolor="#D8D8D8", 
                  xaxis=dict(title="Средняя значимость", showgrid=True, showline=True, 
                             linecolor="#9A9A9A", gridcolor="#F5F5F5"))